### Python API

HyperTS是DataCanvas Automatic Toolkit(DAT)工具链中，依托[Hypernets](https://github.com/DataCanvasIO/Hypernets)研发的关于时间序列的全Pipeline的自动化工具包。它遵循了make_expriment的使用习惯(类似于[HyperGBM](https://github.com/DataCanvasIO/HyperGBM)的API，一个针对于结构化表格数据的AutoML工具)，也符合```scikit-learn```中model API的使用规范。我们可以创造一个```make_expriment```，```run```之后获得pipeline_model, 即一个最终优化完毕的estimator, 然后使用它的```predict```, ```evaluate```, ```plot```去分析未知的数据。

接下来，我们一起动手快速看三个案例指南吧。


1. 预测案例
2. 分类案例
3. 异常检测案例

## 1. 预测案例

### 1.1 准备数据

PS: 如果您对于HyperTS的数据形式存有疑惑，请回看[01_datatypes_for_hyperts.ipynb](https://github.com/DataCanvasIO/HyperTS/blob/main/examples/01_datatypes_for_hyperts.ipynb)中的介绍。

In [ ]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

在划分训练集和测试集中，由于数据存在时间上的先后顺序，为防止信息泄露，我们从整体数据集的后边切分一部分，故```shuffle=False```.

In [ ]:
df = load_network_traffic()
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

在本案例数据集，我们暴露其一些基本信息供参考，具体如下：

- 时间列名称: 'TimeStamp';
- 目标列名称: ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_5', 'Var_6'];
- 协变量列名称: ['HourSin', 'WeekCos', 'CBWD'];
- 时间频率: 'H'.

### 1.2 创建实验

我们通过创建实验```make_experiment```搜索一个时序模型, 并调用```run()```方法来执行实验。

In [ ]:
from hyperts import make_experiment

**注意：** 

在预测任务中，我们必须向```make_experiment```传入参数```timestamp```列名。如果存在协变量，也需要传入```covariables```列名。

因此，在本案例中，我们需要向```make_experiment```传入以下参数：

1.告知其我们现在需要做一个时序预测任务，即```task='forecast'```;

2、告知其数据集的时间列名称，即```timestamp='TimeStamp'```；

3、告知其数据集中协变量列的名称，即```covariables=['HourSin', 'WeekCos', 'CBWD']```;

4、如果想要获得强大的性能表现，还可以修改其他默认的参数，具体可以参考参数介绍。

In [ ]:
experiment = make_experiment(train_data=train_data.copy(),
                             task='forecast',
                             timestamp='TimeStamp',
                             covariables=['HourSin', 'WeekCos', 'CBWD'])
model = experiment.run()

看一看获得最终搜索的模型的参数。

In [ ]:
model.get_pipeline_params()

### 1.3 结果预测

对test data切分X与y, 调用```predict()```方法执行结果预测。

In [ ]:
X_test, y_test = model.split_X_y(test_data.copy())
forecast = model.predict(X_test)
forecast.head()

### 1.4 结果评估

调用```evaluate```方法执行结果评估，便可以观测到各个评估指标下的得分情况。

这里会返回一些默认的指标评分，如果想观测指定指标的评分，可以设置参数```metrics```， 例如metrics=['mae', 'mse', mape_func]。

其中，mape_func可以是自定义的评估函数或者来自于sklearn的评估函数。

In [ ]:
results = model.evaluate(y_true=y_test, y_pred=forecast)
results.head()

### 1.5 可视化

调用```plot()```方法可视化，看一看预测曲线，并与实际的曲线对比一下。

In [ ]:
model.plot(forecast=forecast, actual=test_data, var_id='Var_3', interactive=False)

**注意**

- 这里会显示某一个变量的预测曲线，默认为第一个目标变量；
- 如果为多变量预测，想要观测其他的变量曲线变化的情况，可以修改参数```var_id```, 例如：var_id=2 或者var_id='Var_2'；
- ```plot```可支持交互式可视化通过设置```interactive=False```(默认交互, 需安装plotly);
- 绘制更长期的历史信息，设置参数```history=sub_train_data```.

### 1.6 模型保存

直接采用``save()``方式.

In [ ]:
model.save(model_file="./xxx/xxx/models")

### 1.7 模型加载

In [ ]:
from hyperts.utils.models import load_model

In [ ]:
pipeline_model = load_model(model_file="./xxx/xxx/models/stats_models")

**注意**

加载模型时，需要指定模型保存的文件夹，即"./xxx/xxx/models/stats_models" 而不是"./xxx/xxx/models".

<bar>

<bar>

## 2. 分类案例

### 2.1 准备数据集 

PS: 如果您对于HyperTS的数据形式存有疑惑，请回看[01_datatypes_for_hyperts.ipynb](https://github.com/DataCanvasIO/HyperTS/blob/main/examples/01_datatypes_for_hyperts.ipynb)中的介绍。

In [ ]:
from hyperts.datasets import load_basic_motions
from sklearn.model_selection import train_test_split

In [ ]:
df = load_basic_motions()
train_df, test_df = train_test_split(df, test_size=0.2)

对本案例数据集，为了方便理解，我们暴露一些基本的信息如下：

- 特征列名称: ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_5', 'Var_6'];
- 目标列名称: 'target'.

### 2.2 创建实验

我们通过创建实验```make_experiment```搜索一个时序模型, 并调用```run()```方法来执行实验。

In [ ]:
experiment = make_experiment(train_data=train_df.copy(), task='classification', target='target')

model = experiment.run()

看一看获得最终搜索的模型的参数。

In [ ]:
model.get_pipeline_params()

### 2.3 结果预测

对test data切分X与y, 调用```predict()```方法执行结果预测。

In [ ]:
X_test, y_test = model.split_X_y(test_df.copy())
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

### 2.4 结果评估

调用```evaluate```方法执行结果评估，便可以观测到各个评估指标下的得分情况。


这里会返回一些默认的指标评分，如果想观测指定指标的评分，可以设置参数```metrics```， 例如metrics=['accuracy', 'auc', f1_func]。

其中，mape_func可以是自定义的评估函数或者来自于sklearn的评估函数。

In [ ]:
results = model.evaluate(y_true=y_test, y_pred=y_pred, y_proba=y_proba)
results.head()

### 2.5 模型保存

调用``save_model()``保存模型.

<img src="" width = "500">

In [ ]:
from hyperts.utils.models import save_model

save_model(model=model, model_file="./xxx/xxx/models")

## 3. 异常检测案例

### 3.1 准备数据集

PS: 如果您对于HyperTS的数据形式存有疑惑，请回看[01_datatypes_for_hyperts.ipynb](https://github.com/DataCanvasIO/HyperTS/blob/main/examples/01_datatypes_for_hyperts.ipynb)中的介绍。

In [ ]:
from hyperts.datasets import load_real_known_cause_dataset

In [ ]:
df = load_real_known_cause_dataset()

通常情况下，我们在实际场景中所获得的数据是无标注的。因此，我们将数据中的真实标签分离出来。

In [ ]:
ground_truth = df.pop('anomaly')

拆分训练集和测试集。

In [ ]:
from hyperts.toolbox import temporal_train_test_split

In [ ]:
train_data, test_data = temporal_train_test_split(df, test_horizon=15000)

### 3.2 创建实验

我们通过创建实验```make_experiment```搜索一个时序模型, 并调用```run()```方法来执行实验。

In [ ]:
from hyperts import make_experiment

In [ ]:
experiment = make_experiment(
    train_data=train_data.copy(),
    mode='dl',
    task='detection',
    timestamp='timestamp',
    max_trials=30,
    random_state=2022)

model = experiment.run()

看一看获得最终搜索的模型的参数。

In [ ]:
model.get_pipeline_params()

### 3.3 结果预测

对test data切分X与y, 调用```predict()```和```predict_proba()```方法执行结果预测。

In [ ]:
X_test, _ = model.split_X_y(test_data.copy())

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

### 3.4 结果评估


调用```evaluate```方法执行结果评估，便可以观测到各个评估指标下的得分情况。

In [ ]:
y_test = ground_truth.iloc[-15000:]

In [ ]:
scores = model.evaluate(y_true=y_test, y_pred=y_pred, y_proba=y_proba)
scores

### 3.5 可视化

调用```plot()```方法可视化，看一看异常点的分布和各个时间点的异常程度。

In [ ]:
model.plot(y_pred, actual=test_data, history=train_data, interactive=False)